In [128]:
import keras.losses
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import img_to_array
from tensorflow.keras import layers, Model
from keras.regularizers import l2
import numpy as np
import pandas as pd
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential, Input
from sklearn.utils import class_weight
import datetime

In [72]:
train_triplets = pd.read_csv("train_triplets.txt", sep=' ', header=None, dtype=str)
test_triplets = pd.read_csv("test_triplets.txt", sep=' ', header=None, dtype=str)

In [17]:
def append_ext(fn):
    return fn + '.jpg'

In [120]:
ONE_SET_SIZE = 59515
ABC = ['A', 'B', 'C']

train_triplets_jpg = train_triplets.apply(append_ext)
data_y1 = pd.DataFrame(np.ones((ONE_SET_SIZE)), dtype='str', columns=['label'])
train_triplets_jpg = pd.concat([train_triplets_jpg, data_y1], axis=1)
train_triplets_jpg = train_triplets_jpg.rename(columns={0:'A', 1:'B', 2:'C'}, inplace=False)

test_triplets_jpg = test_triplets.apply(append_ext)
test_triplets_jpg = test_triplets_jpg.rename(columns={0:'A', 1:'B', 2:'C'}, inplace=False)

In [121]:
print(train_triplets_jpg.loc[0,'label'].istype())

AttributeError: 'str' object has no attribute 'istype'

In [153]:
datagen=image.ImageDataGenerator(
    #validation_split=0.2,
    preprocessing_function=preprocess_input
    )

train_generator_A = datagen.flow_from_dataframe(
    train_triplets_jpg,
    directory='./food_porn/food',
    x_col= 'A',
    y_col="label",
    subset="training" ,
    shuffle=True,
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=32,
    seed=69,
    )

train_generator_B = datagen.flow_from_dataframe(
    train_triplets_jpg,
    directory='./food_porn/food',
    x_col= 'B',
    y_col="label",
    subset="training",
    shuffle=True,
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=32,
    seed=69,
    )


train_generator_C = datagen.flow_from_dataframe(
    train_triplets_jpg,
    directory='./food_porn/food',
    x_col= 'C',
    y_col="label",
    subset="training",
    shuffle=True,
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=32,
    seed=69,
    )

""" valid_generator = datagen.flow_from_dataframe(
    train_triplets_jpg,
    directory='./food_porn/food',
    x_col="id",
    y_col="label",
    subset="validation",
    shuffle=True,
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=32,
    seed=69,
    ) """

test_datagen=image.ImageDataGenerator(
    rescale=1./255.,
    preprocessing_function=preprocess_input
    )

test_generator_A = test_datagen.flow_from_dataframe(
    test_triplets_jpg,
    directory='./food_porn/food',
    x_col="A",
    y_col=None,
    shuffle=False,
    class_mode=None,
    target_size=(224, 224),
    batch_size=32,
    seed=69,
    )

test_generator_B = test_datagen.flow_from_dataframe(
    test_triplets_jpg,
    directory='./food_porn/food',
    x_col="B",
    y_col=None,
    shuffle=False,
    class_mode=None,
    target_size=(224, 224),
    batch_size=32,
    seed=69,
    )

test_generator_C = test_datagen.flow_from_dataframe(
    test_triplets_jpg,
    directory='./food_porn/food',
    x_col="C",
    y_col=None,
    shuffle=False,
    class_mode=None,
    target_size=(224, 224),
    batch_size=32,
    seed=69,
    )

Found 59515 validated image filenames belonging to 1 classes.
Found 59515 validated image filenames belonging to 1 classes.
Found 59515 validated image filenames belonging to 1 classes.
Found 59544 validated image filenames.
Found 59544 validated image filenames.
Found 59544 validated image filenames.


In [154]:

train_generator = zip(train_generator_A, train_generator_B, train_generator_C)

In [139]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [140]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [156]:
abc.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 244, 244, 3) 0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           [(None, 244, 244, 3) 0                                            
__________________________________________________________________________________________________
input_33 (InputLayer)           [(None, 244, 244, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Functional)              (None, 512)          14714688    input_31[0][0]                   
                                                                 input_32[0][0]        

In [161]:


inA = Input(shape=(244, 244, 3), dtype='float32')
inB = Input(shape=(244, 244, 3), dtype='float32')
inC = Input(shape=(244, 244, 3), dtype='float32')

outA = vgg16(inA)
outB = vgg16(inB)
outC = vgg16(inC)

combined = layers.Concatenate()([outA, outB, outC])


combined = Dense(units = 1024, activation='relu')(combined)
combined = Dense(units = 1, activation='sigmoid')(combined)

abc = Model(inputs=[inA, inB, inC], outputs=combined)

In [155]:
abc.compile(loss = keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

#fit the network to (for now the un-sparse matrix)
abc.fit(train_generator, epochs=128, batch_size=32, verbose=1)

Epoch 1/128


ValueError: in user code:

    T:\Users\nando\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    T:\Users\nando\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    T:\Users\nando\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    T:\Users\nando\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    T:\Users\nando\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    T:\Users\nando\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
 